In [18]:
# from softcomputing_packages_bu import *
from apyori import apriori
import pandas as pd
import numpy as np
import copy
import networkx as nx
import matplotlib.pyplot as plt

In [19]:
# Load dataset
hc_csv_raw = pd.read_csv('./datasets/2.0-discretized-v2-2-2-with-label-resampled-and-DLP.csv').drop(columns=['Unnamed: 0'])
hc_csv_raw.head()
len(hc_csv_raw)

41673

In [20]:
hc_csv_raw['peak_warning'] = hc_csv_raw['peak_label_pred'].apply(lambda x: 'true' if x == 'peak_warning' else 'false')
hc_csv_raw['no_significant_change'] = hc_csv_raw['peak_label_pred'].apply(lambda x: 'true' if x == 'no_significant_change' else 'false')
hc_csv_raw['lower_than_usual'] = hc_csv_raw['peak_label_pred'].apply(lambda x: 'true' if x == 'lower_than_usual' else 'false')
hc_csv_raw

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,attention_type_label,activation_type_label,peak_warning,no_significant_change,lower_than_usual
0,Very High,Low,Low,Very High,High,Medium,High,Very High,True,march,spring,saturday,no_significant_change,early_attention,cam_1,false,true,false
1,High,Low,High,High,Low,Low,High,High,False,january,winter,thursday,no_significant_change,other,cam_1,false,true,false
2,Very High,High,High,Very High,Medium,Medium,Very High,Very High,False,january,winter,friday,no_significant_change,early_attention,cam_1,false,true,false
3,Low,Low,Low,Low,Medium,Low,Low,Medium,False,february,winter,friday,no_significant_change,late_attention,cam_2,false,true,false
4,Very High,Low,Very High,Very High,Low,Low,Low,Very High,True,december,winter,saturday,no_significant_change,early_attention,cam_1,false,true,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41668,Low,Low,Very High,Low,Low,Low,Low,Medium,True,january,winter,saturday,lower_than_usual,late_attention,cam_2,false,false,true
41669,Medium,Medium,Very High,Medium,High,Medium,Medium,High,False,february,winter,tuesday,lower_than_usual,other,cam_2,false,false,true
41670,Medium,Very High,High,Medium,Low,High,Low,High,False,april,spring,tuesday,lower_than_usual,other,cam_2,false,false,true
41671,Low,Low,Very High,Low,High,High,Medium,Low,True,january,winter,saturday,lower_than_usual,late_attention,cam_2,false,false,true


In [21]:
hc_csv_raw.to_csv('./datasets/2.0-discretized-v2-2-2-with-label-resampled-and-DLP-encoded.csv', index=False)
hc_csv_raw

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,attention_type_label,activation_type_label,peak_warning,no_significant_change,lower_than_usual
0,Very High,Low,Low,Very High,High,Medium,High,Very High,True,march,spring,saturday,no_significant_change,early_attention,cam_1,false,true,false
1,High,Low,High,High,Low,Low,High,High,False,january,winter,thursday,no_significant_change,other,cam_1,false,true,false
2,Very High,High,High,Very High,Medium,Medium,Very High,Very High,False,january,winter,friday,no_significant_change,early_attention,cam_1,false,true,false
3,Low,Low,Low,Low,Medium,Low,Low,Medium,False,february,winter,friday,no_significant_change,late_attention,cam_2,false,true,false
4,Very High,Low,Very High,Very High,Low,Low,Low,Very High,True,december,winter,saturday,no_significant_change,early_attention,cam_1,false,true,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41668,Low,Low,Very High,Low,Low,Low,Low,Medium,True,january,winter,saturday,lower_than_usual,late_attention,cam_2,false,false,true
41669,Medium,Medium,Very High,Medium,High,Medium,Medium,High,False,february,winter,tuesday,lower_than_usual,other,cam_2,false,false,true
41670,Medium,Very High,High,Medium,Low,High,Low,High,False,april,spring,tuesday,lower_than_usual,other,cam_2,false,false,true
41671,Low,Low,Very High,Low,High,High,Medium,Low,True,january,winter,saturday,lower_than_usual,late_attention,cam_2,false,false,true


In [67]:
# Sampling (for Development)
n_sample = 30000
hc_csv = hc_csv_raw.sample(frac=1, random_state=42)
hc_csv = hc_csv.iloc[:n_sample]
hc_csv.head()

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,peak_warning,no_significant_change,lower_than_usual
35103,Medium,Very High,Very High,Medium,Medium,Low,High,High,True,april,spring,sunday,lower_than_usual,False,False,True
8049,Low,High,Very High,Low,Low,Low,Low,Medium,False,march,spring,tuesday,no_significant_change,False,True,False
22355,Low,Medium,Medium,Low,Medium,High,Medium,Low,False,september,autumn,friday,peak_warning,True,False,False
21779,Medium,Very High,Medium,Medium,Very High,Very High,High,Medium,True,june,summer,saturday,peak_warning,True,False,False
40302,Low,Medium,High,Low,High,High,Medium,Medium,False,march,spring,wednesday,lower_than_usual,False,False,True


In [68]:
# Preprocess dataset
for c in range(8):
    print(c)
    print("[%d] Column Preprocessing: %s" %(c, hc_csv.columns[c]))
    for r in range(hc_csv.shape[0]):
        if(hc_csv.iloc[r,c]=='Very High'):
            hc_csv.iloc[r,c] = hc_csv.columns[c]+' 75-100%'
        elif(hc_csv.iloc[r,c]=='High'):
            hc_csv.iloc[r,c] = hc_csv.columns[c]+' 50-75%'
        elif(hc_csv.iloc[r,c]=='Medium'):
            hc_csv.iloc[r,c] = hc_csv.columns[c]+' 25-50%'
        elif(hc_csv.iloc[r,c]=='Low'):
            # hc_csv.iloc[r,c] = '-1'
            hc_csv.iloc[r,c] = hc_csv.columns[c]+' 0-25%'
        else:
            hc_csv.iloc[r,c] = '-1'
print("Done.")

0
[0] Column Preprocessing: global_active_power
1
[1] Column Preprocessing: global_reactive_power
2
[2] Column Preprocessing: voltage
3
[3] Column Preprocessing: global_intensity
4
[4] Column Preprocessing: kitchen
5
[5] Column Preprocessing: laundry
6
[6] Column Preprocessing: climate_control
7
[7] Column Preprocessing: other
Done.


In [69]:
for c in hc_csv.columns:
  if c == 'weekend':
    for r in range(hc_csv.shape[0]):
      if hc_csv.iloc[r,list(hc_csv.columns).index('weekend')] == True:
        hc_csv.iloc[r,list(hc_csv.columns).index('weekend')] = 'weekend'
      else:
        hc_csv.iloc[r,list(hc_csv.columns).index('weekend')] = 'weekday'
      # break

C:\Users\sclab\AppData\Local\Temp\ipykernel_33248\1856367701.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'weekend' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  hc_csv.iloc[r,list(hc_csv.columns).index('weekend')] = 'weekend'


In [70]:
hc_csv.sample(5)

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,peak_warning,no_significant_change,lower_than_usual
28950,global_active_power 0-25%,global_reactive_power 0-25%,voltage 50-75%,global_intensity 25-50%,kitchen 50-75%,laundry 25-50%,climate_control 25-50%,other 25-50%,weekend,april,spring,sunday,lower_than_usual,False,False,True
17803,global_active_power 25-50%,global_reactive_power 75-100%,voltage 50-75%,global_intensity 25-50%,kitchen 25-50%,laundry 75-100%,climate_control 25-50%,other 50-75%,weekend,june,summer,sunday,peak_warning,True,False,False
13194,global_active_power 25-50%,global_reactive_power 0-25%,voltage 75-100%,global_intensity 25-50%,kitchen 50-75%,laundry 50-75%,climate_control 25-50%,other 50-75%,weekday,december,winter,thursday,no_significant_change,False,True,False
38779,global_active_power 75-100%,global_reactive_power 25-50%,voltage 50-75%,global_intensity 75-100%,kitchen 0-25%,laundry 0-25%,climate_control 75-100%,other 50-75%,weekday,february,winter,friday,lower_than_usual,False,False,True
8814,global_active_power 25-50%,global_reactive_power 50-75%,voltage 50-75%,global_intensity 25-50%,kitchen 75-100%,laundry 50-75%,climate_control 25-50%,other 25-50%,weekend,october,autumn,sunday,no_significant_change,False,True,False


In [71]:
# targets_only = ['peak_label_pred', 'no_significant_change', 'lower_than_usual']

In [77]:
hc_csv_temp = copy.copy(hc_csv)
hc_csv_temp.drop(columns=['month_name', 'day_name', 'peak_label_pred', 'peak_warning', 'lower_than_usual'], inplace=True)

In [78]:
for i in range(len(hc_csv_temp)):
    if hc_csv_temp.iloc[i,-1] == True:
        hc_csv_temp.iloc[i,-1] = 'True'
    else:
        hc_csv_temp.iloc[i,-1] = 'False'
    # break

C:\Users\sclab\AppData\Local\Temp\ipykernel_33248\3489011138.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  hc_csv_temp.iloc[i,-1] = 'False'


In [82]:
hc_csv_temp.head(4)

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,season_name,no_significant_change
35103,global_active_power 25-50%,global_reactive_power 75-100%,voltage 75-100%,global_intensity 25-50%,kitchen 25-50%,laundry 0-25%,climate_control 50-75%,other 50-75%,weekend,spring,False
8049,global_active_power 0-25%,global_reactive_power 50-75%,voltage 75-100%,global_intensity 0-25%,kitchen 0-25%,laundry 0-25%,climate_control 0-25%,other 25-50%,weekday,spring,True
22355,global_active_power 0-25%,global_reactive_power 25-50%,voltage 25-50%,global_intensity 0-25%,kitchen 25-50%,laundry 50-75%,climate_control 25-50%,other 0-25%,weekday,autumn,False
21779,global_active_power 25-50%,global_reactive_power 75-100%,voltage 25-50%,global_intensity 25-50%,kitchen 75-100%,laundry 75-100%,climate_control 50-75%,other 25-50%,weekend,summer,False


In [80]:
# hc_csv to list
hc_list = []
for r in range(hc_csv_temp.shape[0]):
    rule_buffer = []
    for c in range(len(hc_csv_temp.iloc[r])):
        if(hc_csv_temp.iloc[r,c]!='-1'):
            rule_buffer.append(hc_csv_temp.iloc[r,c])
    hc_list.append(rule_buffer)

In [81]:
hc_list

[['global_active_power 25-50%',
  'global_reactive_power 75-100%',
  'voltage 75-100%',
  'global_intensity 25-50%',
  'kitchen 25-50%',
  'laundry 0-25%',
  'climate_control 50-75%',
  'other 50-75%',
  'weekend',
  'spring',
  'False'],
 ['global_active_power 0-25%',
  'global_reactive_power 50-75%',
  'voltage 75-100%',
  'global_intensity 0-25%',
  'kitchen 0-25%',
  'laundry 0-25%',
  'climate_control 0-25%',
  'other 25-50%',
  'weekday',
  'spring',
  'True'],
 ['global_active_power 0-25%',
  'global_reactive_power 25-50%',
  'voltage 25-50%',
  'global_intensity 0-25%',
  'kitchen 25-50%',
  'laundry 50-75%',
  'climate_control 25-50%',
  'other 0-25%',
  'weekday',
  'autumn',
  'False'],
 ['global_active_power 25-50%',
  'global_reactive_power 75-100%',
  'voltage 25-50%',
  'global_intensity 25-50%',
  'kitchen 75-100%',
  'laundry 75-100%',
  'climate_control 50-75%',
  'other 25-50%',
  'weekend',
  'summer',
  'False'],
 ['global_active_power 0-25%',
  'global_reactive_po

In [83]:
# Association Rule Mining (ARM) (Caution: extremly slow)
association_rules = list(apriori(hc_list, min_support=0.05, min_confidence=0.2))
print('Rule Mined: %d' %(len(association_rules)))

Rule Mined: 1479


In [85]:
# Rules to numpy structure
association_rules_np = []
for r in range(len(association_rules)):
    rule_buffer = []
    association_rule = association_rules[r]
    items = [x for x in association_rule[0]]
    n_items = len(items)
    support = association_rule[1]
    confidence = association_rule[2][0][2]
    lift = association_rule[2][0][3]
    rule_buffer.append(n_items), rule_buffer.append(np.array(items)), rule_buffer.append(round(support, 4)), rule_buffer.append(round(confidence, 4)), rule_buffer.append(round(lift, 4))
    print(rule_buffer)
    association_rules_np.append(rule_buffer)
# Fix later
# association_rules_np = np.array(association_rules_np)
# print(association_rules_np)
# np.save('./association_rules', association_rules_np)
# Length, Rule, Support, Confidence, Lift

[1, array(['False'], dtype='<U5'), 0.666, 0.666, 1.0]
[1, array(['True'], dtype='<U4'), 0.334, 0.334, 1.0]
[1, array(['autumn'], dtype='<U6'), 0.2464, 0.2464, 1.0]
[1, array(['climate_control 0-25%'], dtype='<U21'), 0.3145, 0.3145, 1.0]
[1, array(['climate_control 25-50%'], dtype='<U22'), 0.2476, 0.2476, 1.0]
[1, array(['climate_control 50-75%'], dtype='<U22'), 0.2261, 0.2261, 1.0]
[1, array(['climate_control 75-100%'], dtype='<U23'), 0.2118, 0.2118, 1.0]
[1, array(['global_active_power 0-25%'], dtype='<U25'), 0.3074, 0.3074, 1.0]
[1, array(['global_active_power 25-50%'], dtype='<U26'), 0.2678, 0.2678, 1.0]
[1, array(['global_active_power 50-75%'], dtype='<U26'), 0.2061, 0.2061, 1.0]
[1, array(['global_active_power 75-100%'], dtype='<U27'), 0.2188, 0.2188, 1.0]
[1, array(['global_intensity 0-25%'], dtype='<U22'), 0.3144, 0.3144, 1.0]
[1, array(['global_intensity 25-50%'], dtype='<U23'), 0.2603, 0.2603, 1.0]
[1, array(['global_intensity 50-75%'], dtype='<U23'), 0.2059, 0.2059, 1.0]
[1, 

In [86]:
# Filter by rhs=='Power Pattern *'
filter_RHS_ECP = True
if(filter_RHS_ECP):
    association_rules_np_rhs = []
    for r in range(len(association_rules_np)):
        # rhs = association_rules_np[r, 1][-1] # old
        rhs = association_rules_np[r][1][-1]
        if(rhs=='True' or rhs=='False'):
            association_rules_np_rhs.append(association_rules_np[r])
    association_rules_np_rhs = np.array(association_rules_np_rhs, dtype=object)
else:
    association_rules_np_rhs = association_rules_np
print('Rule Filtered: %d' %(len(association_rules_np_rhs)))

Rule Filtered: 104


In [87]:
# Sort rules by confidence
association_rules_np_rhs_sorted = association_rules_np_rhs[association_rules_np_rhs[:, 3].argsort()][::-1]
print(len(association_rules_np_rhs_sorted))
print(association_rules_np_rhs_sorted[:10])

# Length, Rule, Support, Confidence, Lift

104
[[1 array(['False'], dtype='<U5') 0.666 0.666 1.0]
 [3 array(['voltage 0-25%', 'weekday', 'False'], dtype='<U13') 0.1229
  0.5173 1.1079]
 [3 array(['global_intensity 75-100%', 'weekday', 'False'], dtype='<U24')
  0.1099 0.5007 1.0722]
 [3 array(['summer', 'weekday', 'False'], dtype='<U7') 0.1215 0.5003
  1.0715]
 [3 array(['other 0-25%', 'weekday', 'False'], dtype='<U11') 0.1307
  0.4997 1.0701]
 [3
  array(['global_reactive_power 50-75%', 'weekday', 'False'], dtype='<U28')
  0.1149 0.4723 1.0115]
 [2 array(['weekday', 'False'], dtype='<U7') 0.467 0.467 1.0]
 [3 array(['laundry 25-50%', 'weekday', 'False'], dtype='<U14') 0.1146
  0.4566 0.9777]
 [3 array(['voltage 25-50%', 'weekday', 'False'], dtype='<U14') 0.1032
  0.4539 0.9721]
 [3 array(['laundry 50-75%', 'weekday', 'False'], dtype='<U14') 0.1103
  0.4511 0.966]]


In [99]:
# Filter by rhs=='No change - True'
top_k = 10
filtered_rules = []
association_rules_np_rhs_buffer = []
for r in range(len(association_rules_np_rhs_sorted)):
    rhs = association_rules_np_rhs_sorted[r, 1][-1]
    if(rhs=='True' and association_rules_np_rhs_sorted[r, 0]>1):
        association_rules_np_rhs_buffer.append(association_rules_np_rhs_sorted[r])
association_rules_np_rhs_buffer = np.array(association_rules_np_rhs_buffer)
# print(association_rules_np_rhs_buffer[:top_k])
for rule in association_rules_np_rhs_buffer:
    # print(rule)
    if top_k < 1:
        break
    if rule[0] == 0 or rule[-1] < 1.1:
        # print(rule)
        # print(rule)
        continue
    else:
        print(rule)
        filtered_rules.extend(rule[1].tolist())
        top_k = top_k - 1

print('unique vars')
set(filtered_rules) - {'True'}

[2 array(['global_active_power 25-50%', 'True'], dtype='<U26') 0.1059
 0.317 1.184]
[2 array(['global_intensity 25-50%', 'True'], dtype='<U23') 0.1059 0.317
 1.2181]
[3
 array(['global_intensity 25-50%', 'global_active_power 25-50%', 'True'],
       dtype='<U26')
 0.0977 0.2925 1.2111]
[2 array(['voltage 50-75%', 'True'], dtype='<U14') 0.0943 0.2823 1.209]
[3 array(['other 50-75%', 'weekday', 'True'], dtype='<U12') 0.0645 0.2763
 1.129]
[3 array(['laundry 50-75%', 'weekday', 'True'], dtype='<U14') 0.0663
 0.2712 1.1085]
[2 array(['other 50-75%', 'True'], dtype='<U12') 0.0903 0.2704 1.1577]
[2 array(['climate_control 50-75%', 'True'], dtype='<U22') 0.0859 0.2571
 1.1368]
[3 array(['global_intensity 25-50%', 'weekday', 'True'], dtype='<U23')
 0.0786 0.2353 1.2786]
[3 array(['global_active_power 25-50%', 'weekday', 'True'], dtype='<U26')
 0.0784 0.2348 1.2474]
unique vars


{'climate_control 50-75%',
 'global_active_power 25-50%',
 'global_intensity 25-50%',
 'laundry 50-75%',
 'other 50-75%',
 'voltage 50-75%',
 'weekday'}

In [136]:
association_rules_np_rhs_buffer

array([[2, array(['global_intensity 75-100%', 'True'], dtype='<U24'),
        0.1625, 0.489, 2.2276],
       [2, array(['climate_control 75-100%', 'True'], dtype='<U23'),
        0.1426, 0.4293, 2.0265],
       [2, array(['other 75-100%', 'True'], dtype='<U13'), 0.1405,
        0.4228, 1.9687],
       [2, array(['voltage 0-25%', 'True'], dtype='<U13'), 0.1361,
        0.4095, 1.7243],
       [2, array(['summer', 'True'], dtype='<U6'), 0.1168, 0.3516,
        1.4476],
       [3,
        array(['global_intensity 75-100%', 'other 75-100%', 'True'], dtype='<U24'),
        0.1151, 0.3463, 2.2924],
       [2, array(['kitchen 75-100%', 'True'], dtype='<U15'), 0.1129,
        0.3398, 1.5258],
       [2,
        array(['global_reactive_power 75-100%', 'True'], dtype='<U29'),
        0.1106, 0.3329, 1.4004],
       [2, array(['voltage 25-50%', 'True'], dtype='<U14'), 0.1018,
        0.3064, 1.3481],
       [2, array(['weekend', 'True'], dtype='<U7'), 0.1016, 0.3059,
        1.0609],
       [2, a

In [138]:
hc_csv_raw['peak_warning'] = hc_csv_raw['peak_label_pred'].apply(lambda x: 'true' if x == 'peak_warning' else 'false')
hc_csv_raw['no_significant_change'] = hc_csv_raw['peak_label_pred'].apply(lambda x: 'true' if x == 'no_significant_change' else 'false')
hc_csv_raw['lower_than_usual'] = hc_csv_raw['peak_label_pred'].apply(lambda x: 'true' if x == 'lower_than_usual' else 'false')
hc_csv_raw

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,peak_warning,no_significant_change,lower_than_usual
0,Very High,Low,Low,Very High,High,Medium,High,Very High,True,march,spring,saturday,no_significant_change,false,true,false
1,High,Low,High,High,Low,Low,High,High,False,january,winter,thursday,no_significant_change,false,true,false
2,Very High,High,High,Very High,Medium,Medium,Very High,Very High,False,january,winter,friday,no_significant_change,false,true,false
3,Low,Low,Low,Low,Medium,Low,Low,Medium,False,february,winter,friday,no_significant_change,false,true,false
4,Very High,Low,Very High,Very High,Low,Low,Low,Very High,True,december,winter,saturday,no_significant_change,false,true,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41668,Low,Low,Very High,Low,Low,Low,Low,Medium,True,january,winter,saturday,lower_than_usual,false,false,true
41669,Medium,Medium,Very High,Medium,High,Medium,Medium,High,False,february,winter,tuesday,lower_than_usual,false,false,true
41670,Medium,Very High,High,Medium,Low,High,Low,High,False,april,spring,tuesday,lower_than_usual,false,false,true
41671,Low,Low,Very High,Low,High,High,Medium,Low,True,january,winter,saturday,lower_than_usual,false,false,true


save

In [132]:
# hc_csv_raw.to_csv('./datasets/2.0-discretized-v2-3-peak-label-encoded.csv', index=False)